In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer

In [4]:
df = pd.read_csv('email.csv')  # Replace with your dataset file
df['Label'] = df['Category'].map({'ham':0, "spam":1})
print(df.head)



train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['Message'], df['Label'], test_size = 0.2, random_state = 42
)



<bound method NDFrame.head of             Category                                            Message  Label
0                ham  Go until jurong point, crazy.. Available only ...    0.0
1                ham                      Ok lar... Joking wif u oni...    0.0
2               spam  Free entry in 2 a wkly comp to win FA Cup fina...    1.0
3                ham  U dun say so early hor... U c already then say...    0.0
4                ham  Nah I don't think he goes to usf, he lives aro...    0.0
...              ...                                                ...    ...
5568             ham               Will ü b going to esplanade fr home?    0.0
5569             ham  Pity, * was in mood for that. So...any other s...    0.0
5570             ham  The guy did some bitching but I acted like i'd...    0.0
5571             ham                         Rofl. Its true to its name    0.0
5572  {"mode":"full"                                    isActive:false}    NaN

[5573 rows x 3 column

In [ ]:
# Tokenize Messages
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


#Tokenize the text data
train_encodings = tokenizer(list(train_texts), truncation = True, padding=True, max_length=128)
val_encodings = tokenizer(list(val_texts), truncation=True, padding = True, max_length=128)

In [2]:
import torch

class MessageDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return{
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': self.labels[idx]
        }
        
    
    
    
    
    
    